# Introduction

[<img align="center" src="https://colab.research.google.com/assets/colab-badge.svg" />](https://colab.research.google.com/github/marshmellow77/automated-prompt-engineering/blob/main/automated-prompt-engineering.ipynb)


This notebook demonstrates how to use Google's Gemini model to automate prompt engineering.

Prompt engineering is a powerful way to improve the responses og large language models (LLMs). Bit it is also a manual, tedious, iterative process and it quickly accumulates technical debt and waste since each handcrafted prompt is specific to a model (and its version) as well as the task at hand.

In this notebook we will learn how to use the DSPy library to autonomously and automatically create prompts that are optimised for a specific model and the task at hand.


# Manual Prompt Engineering

Manual prompt engineering is very tedious - let's look at an example where we carefully handcraft a prompt for our task and model.

## Setup

In [92]:
# As of 3 April 2024, VertexAI is not yet integrated into DSPy. But there already exists a PR for it which we can leverage.
!pip install -U git+https://github.com/marshmellow77/dspy.git@seedstart-random-search#egg=dspy-ai

72645.01s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
  Cloning https://github.com/marshmellow77/dspy.git (to revision seedstart-random-search) to /private/var/folders/3d/6syzgsd52fjfp7lpbwxtjfnc016sv0/T/pip-install-llm9vak2/dspy-ai_864c2099b96f4f53a8daac970ae230b1
  Running command git clone --filter=blob:none --quiet https://github.com/marshmellow77/dspy.git /private/var/folders/3d/6syzgsd52fjfp7lpbwxtjfnc016sv0/T/pip-install-llm9vak2/dspy-ai_864c2099b96f4f53a8daac970ae230b1
  Running command git checkout -b seedstart-random-search --track origin/seedstart-random-search
  Switched to a new branch 'seedstart-random-search'
  branch 'seedstart-random-search' set up to track 'origin/seedstart-random-search'.
  Resolved https://github.com/marshmellow77/dspy.git to commit 41718f802655c1d7359cb498efaba6659c07450b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  

In [66]:
!pip install --upgrade google-cloud-aiplatform
!pip install Jinja2

  Obtaining dependency information for google-cloud-aiplatform from https://files.pythonhosted.org/packages/37/79/c8bd845c85a097cee8d80fad6551db0fabb9a93c9aaf6eb955422d675cbb/google_cloud_aiplatform-1.46.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for docstring-parser<1 from https://files.pythonhosted.org/packages/d5/7c/e9fcff7623954d86bdc17782036cbf715ecab1bec4847c008557affe1ca8/docstring_parser-0.16-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 806.5 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.44.0
    Uninstalling google-cloud-aiplatform-1.44.0:
      Successfully uninstalled google-cloud-aiplatform-1.44.0

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os
import sys

IS_COLAB = "google.colab" in sys.modules
if not IS_COLAB:
    raise ValueError("This notebook should be run using Google Colab.")

if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

In [29]:
import vertexai

project_id = "cloud-llm-preview1"
vertexai.init(project=project_id)

In [30]:
from vertexai.generative_models import GenerativeModel

gemini_pro = GenerativeModel("gemini-1.0-pro")

## Zero shot attempt

Let's first try to use Gemini Pro for a mathematical text question

In [31]:
prompt = """Given the fields `question`, produce the fields `answer`.

Question: Heather is going to sew 150 aprons that are to be used for a kiddie crew program.
She already was able to sew 13 aprons, and today, she sewed three times as many aprons.
How many aprons should she sew tomorrow if she wants to sew half of the remaining number of aprons needed?

Answer:"""

# The correct answer is 49.

In [32]:
config = {"temperature": 0.1}

In [33]:
response = gemini_pro.generate_content(contents=prompt, generation_config=config)
print(response.text)

Answer: 69


We can see that Gemini Pro got this one wrong. Let's use best practices including Chain of thought and few shot prompting to improve Gemini's performance!

## Few shot prompting with Chain of Thought

In [34]:
prompt = """Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: <Question>
Rationale: Let's think step by step ...
Answer: <Answer>

---

Question: A gumball machine has red, green, and blue gumballs. The machine has half as many blue gumballs as red gumballs.
For each blue gumball, the machine has 4 times as many green gumballs. If the machine has 16 red gumballs how many gumballs are in the machine?
Rationale: Let's think step by step.
First, we can find the number of blue gumballs in the machine.
Since the machine has half as many blue gumballs as red gumballs, and there are 16 red gumballs, there must be 16 / 2 = 8 blue gumballs.
Next, we can find the number of green gumballs in the machine.
Since the machine has 4 times as many green gumballs as blue gumballs, there must be 8 x 4 = 32 green gumballs.
Finally, we can add up the number of red, blue, and green gumballs to find the total number of gumballs in the machine: 16 + 8 + 32 = 56.
Answer: 56

---

Question: Rachel makes $12.00 as a waitress in a coffee shop. In one hour, she serves 20 different people and they all leave her a $1.25 tip. How much money did she make in that hour?
Rationale: Let's think step by step.
First, we need to find out how much money Rachel made from tips. She served 20 people and each person left her a $1.25 tip, so she made 20 * $1.25 = $25.00 in tips.
Next, we need to add her hourly wage to the money she made from tips to find out how much money she made in total. She made $12.00 per hour, so in one hour she made $12.00 + $25.00 = $37.00.
Answer: 37

---

Question: Heather is going to sew 150 aprons that are to be used for a kiddie crew program. She already was able to sew 13 aprons, and today, she sewed three times as many aprons. How many aprons should she sew tomorrow if she wants to sew half of the remaining number of aprons needed?
Rationale:"""

In [35]:
response = gemini_pro.generate_content(contents=prompt, generation_config=config)
print(response.text)

Let's think step by step.
First, we need to find out how many aprons Heather has left to sew. She needs to sew 150 aprons in total, and she has already sewn 13 aprons, so she has 150 - 13 = 137 aprons left to sew.
Next, we need to find out how many aprons Heather sewed today. She sewed three times as many aprons as she already had, so she sewed 3 * 13 = 39 aprons today.
Now, we need to find out how many aprons Heather has left to sew after today. She has 137 aprons left to sew, and she sewed 39 aprons today, so she has 137 - 39 = 98 aprons left to sew.
Finally, we need to find out how many aprons Heather should sew tomorrow if she wants to sew half of the remaining number of aprons needed. She has 98 aprons left to sew, and she wants to sew half of them, so she should sew 98 / 2 = 49 aprons tomorrow.
Answer: 49


Nice, this worked!

Now we have a good a good prompt for our model and the task at hand (mathematical text questions). But there are a few issues:
* Our prompt works well on our model, but what if we want to use another model or another version (e.g. Gemini Ultra of Gemini 1.5)? Will it still work for those models?
* We had to develop a few examples, and especially coming up with the rationale for each example was tedious

The question is, could we automate this process so that next time we need to repeat this exercise we can just automatically create few shot examples that are optimised for our model and the task at hand?

# Automated prompt engineering with DSPy

DSPy is a library that allows us to automate this process. Let's see how it works.

## Setup

In [36]:
import dspy

In [37]:
dspy_gemini_pro = dspy.GoogleVertexAI(
    "gemini-1.0-pro",
    temperature=0,
)

dspy.settings.configure(lm=dspy_gemini_pro)

## Dataset

We will use the [GSM8K dataset](https://paperswithcode.com/dataset/gsm8k) which consists of inguistically diverse grade school math word problems.

In [38]:
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric

gms8k = GSM8K()

100%|██████████| 1319/1319 [00:00<00:00, 66646.03it/s]


In [39]:
train, val, test = gms8k.train[:60], gms8k.dev[:20], gms8k.test[:20]

In [40]:
train[0]

Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 'gold_reasoning': "Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}) (input_keys={'question'})

In [41]:
train[0].gold_reasoning

"Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items."

We can see that the dataset has a field `gold_resoning`, which already provides reasoning. Since this is what we want to automate, let's delete these for the training and validation datasets.

In [42]:
# Iterate through datasets and modify the dicts
for dataset in [train, val]:
    for example in dataset:
        example["gold_reasoning"] = ""

In [43]:
train[0].gold_reasoning

''

## Defining the signature

Signatures allow you tell the LM what it needs to do, rather than specify how we should ask the LM to do it.

In [44]:
class GSM8KSignature(dspy.Signature):
    """Answer math problems with numbers or short phrases."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="Usually a number or short phrase.")

Now we can use this signature to run a test with Gemini.

In [45]:
generate_answer = dspy.Predict(GSM8KSignature)
pred = generate_answer(question=test[0].question)

print(f"Question: {test[0].question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Actual Answer: {test[0].answer}")

Question: Amber, Micah, and Ahito ran 52 miles in total. Amber ran 8 miles. Micah ran 3.5 times what Amber ran. How many miles did Ahito run?
Predicted Answer: 26.5 miles
Actual Answer: 16


In [46]:
dspy_gemini_pro.inspect_history(n=1)





Answer math problems with numbers or short phrases.

---

Follow the following format.

Question: ${question}
Answer: Usually a number or short phrase.

---

Question: Amber, Micah, and Ahito ran 52 miles in total. Amber ran 8 miles. Micah ran 3.5 times what Amber ran. How many miles did Ahito run?
Answer:26.5 miles





Similar to above Gemini didn't get this one right. Let's evaluate Gemini of the test dataset to establish a baseline.

## Model evaluation with zero shot

To run the evaluation programmatically we define a DSPy module These modules abstract a prompting technique (like chain of thought or ReAct). Crucially, they are generalized to handle any DSPy Signature.

In [47]:
class GSM8KModule(dspy.Module):
    def __init__(self):
        super().__init__()
        # here we use the dspy.Predict module which uses zero shot prompting to generate answers
        self.prog = dspy.Predict(GSM8KSignature) 

    def forward(self, question):
        return self.prog(question=question)

In [48]:
gsm8k_zero_shot = GSM8KModule()

In [49]:
from dspy.evaluate import Evaluate

NUM_THREADS = 4 # number of threads to use for parallel processing
evaluate = Evaluate(
    devset=test, # the test set
    metric=gsm8k_metric, # the metric to use -> this will convert responses to integers to compare with the gold answers
    num_threads=NUM_THREADS,
    display_progress=True,
    display_table=20, # how many rows to display
)

In [50]:
evaluate(gsm8k_zero_shot)

Average Metric: 7 / 20  (35.0): 100%|██████████| 20/20 [00:02<00:00,  6.96it/s]

Average Metric: 7 / 20  (35.0%)



/Users/heikohotz/.pyenv/versions/3.11.7/envs/venv-ape/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['False' '✔️ [True]' '✔️ [True]' 'False' 'False' 'False' 'False' 'False'
 'False' 'False' '✔️ [True]' '✔️ [True]' 'False' '✔️ [True]' 'False'
 'False' '✔️ [True]' 'False' '✔️ [True]' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,gold_reasoning,example_answer,pred_answer,gsm8k_metric
0,"Amber, Micah, and Ahito ran 52 miles in total. Amber ran 8 miles. Micah ran 3.5 times what Amber ran. How many miles did Ahito...",Amber ran <<8=8>>8 miles. Micah ran 3.5 * 8 miles = <<3.5*8=28>>28 miles Ahito ran the rest = 52 - 8 - 28 = <<52-8-28=16>>16...,16,26.5 miles,False
1,"Miguel uses 2 pads of paper a week for his drawing. If there are 30 sheets of paper on a pad of paper, how many...","Miguel uses 30 x 2 = <<30*2=60>>60 sheets of paper every week. Therefore, he uses 60 x 4 = <<60*4=240>>240 sheets of paper every month.",240,240 sheets,✔️ [True]
2,"At a certain grade level, three-fourths of students have a desktop computer at home. If 20 students do not have desktop computers, how many students...","Twenty students represent 1 - 3/4 = 1/4 of the students at that level. So, there are 4 x 20 students = <<4*20=80>>80 students at...",80,80,✔️ [True]
3,Comet Halley orbits the sun every 75 years. Bill's dad saw the Comet when he was 30 years old. Bill saw the comet a second...,Bill saw the Comet for the second time when he was 30 years * 3= <<30*3=90>>90 years old. Comet Halley can be seen every 75...,15,60,False
4,Tom plants 10 trees a year. Every year he also chops down 2 trees a year. He starts with 50 trees. After 10 years 30%...,He gets 10-2=<<10-2=8>>8 new trees a year After 10 years he has 8*10=<<8*10=80>>80 new trees So he had 80+50=<<80+50=130>>130 trees He lost 130*.3=<<130*.3=39>>39 trees That...,91,35,False
5,"John picks 4 bananas on Wednesday. Then he picks 6 bananas on Thursday. On Friday, he picks triple the number of bananas he did on...","Combining Wednesday and Thursday, John has 4 bananas + 6 bananas = <<4+6=10>>10 bananas. On Friday, he picks 3 * 4 bananas = <<3*4=12>>12 bananas....",22,18 bananas,False
6,"Peyton scheduled after-work activities of a one hour yoga class on Monday, a cooking class that lasts three times as long as Monday’s yoga on...",Peyton’s cooking class will last 3 * 1 = <<3*1=3>>3 hours. The museum tour will take 3 / 2 = 1 1/2 hours. Peyton’s after-work...,8,6.5 hours,False
7,"Ben has 4 tubes of blue paint and 3 tubes of yellow paint. Jasper has half as many tubes of blue paint as Ben, and...",Jasper has 4/2= <<4/2=2>>2 tubes of blue paint Jasper has 3*3=<<3*3=9>>9 tubes of yellow paint Jasper has a total of 2+9 =<<2+9=11>>11 tubes of paint,11,10,False
8,"Elaina is holding the final concert in her tour. To celebrate her final concert, she makes the concert twice as long as her usual concerts....","The concert, minus the encore, lasted for 65-minute concert – 15-minute encore = <<65-15=50>>50 minutes. This is twice as long as her usual concerts so...",25,30 minutes,False
9,"Hannah slips on a banana peel and breaks her arm. The doctor charges her $200 for the cast, $300/hour for a 30-minute visit, $4/pill for...",First find the length of the visit in hours: 30 minutes / 60 minutes/hour = <<30/60=.5>>.5 hours Then find the total cost of the visit:...,482,$804,False


35.0

# Bootstrapping few shot examples

Now we will leverage Gemini Ultra to bootstrap few shot examples which will (hopefully) improve Gemini Pro's performance on the test dataset. With Gemini Ultra we will create a few reasoning examples which we can include in the prompt that we will eventually send to Gemini Pro. Ultra will produce a few candidates and test them on a validation dataset using the `gsm8k_metric`, i.e. the metric we want to optimise for. Once the best candidates have been identified these examples will then be used to create a few shot prompt.

First we define a Chain of Thought module:

In [51]:
class ZeroShotCoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(
            GSM8KSignature,
        )

    def forward(self, question):
        return self.prog(question=question)

In [52]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

Now we can start the bootstrapping:

In [100]:
from datetime import datetime

RUN_FROM_SCRATCH = True
bootstrapped_demos = 8 # how many examples are randomly being used from the training dataset
labeled_demos = 3 # how many examples will be in final prompt
candidate_programs = 2 # how many candidates will be created and evaluated (equivalent to epochs)
teacher_model_id = "gemini-1.0-ultra" 

if RUN_FROM_SCRATCH:
    dspy_gemini_ultra = dspy.GoogleVertexAI(
        teacher_model_id,
        temperature=0,
    )
    dspy.settings.configure(lm=dspy_gemini_ultra, timeout=0)
    config = dict(
        max_bootstrapped_demos=bootstrapped_demos,
        max_labeled_demos=labeled_demos,
        num_candidate_programs=candidate_programs,
        num_threads=4,
        stop_at_score=100.0,
    )
    bootstrap_optimizer = BootstrapFewShotWithRandomSearch(
        metric=gsm8k_metric, **config
    )
    cot_fewshot = bootstrap_optimizer.compile(ZeroShotCoT(), trainset=train, valset=val, seed_start=0)

    # save the bootstrap demonstrations for future use
    timestamp_str = datetime.now().strftime("%Y%m%d-%H%M%S")
    filename = f"{timestamp_str}_{teacher_model_id}_{bootstrapped_demos}_{labeled_demos}_{candidate_programs}.json"
    cot_fewshot.save(filename)
else:
    cot_fewshot = ZeroShotCoT()
    cot_fewshot.load("20240403-173150_gemini-1.0-ultra_8_3_2.json")

Going to sample between 1 and 8 traces per predictor.
Will attempt to train 2 candidate sets.


 15%|█▌        | 9/60 [00:42<04:00,  4.71s/it]


Bootstrapped 7 full traces after 10 examples in round 0.


Average Metric: 19 / 20  (95.0): 100%|██████████| 20/20 [00:23<00:00,  1.18s/it] 


Average Metric: 19 / 20  (95.0%)
Score: 95.0 for set: [7]
New best score: 95.0 for seed 0
Scores so far: [95.0]
Best score: 95.0


  5%|▌         | 3/60 [00:12<04:04,  4.29s/it]


Bootstrapped 3 full traces after 4 examples in round 0.


Average Metric: 18 / 20  (90.0): 100%|██████████| 20/20 [00:23<00:00,  1.17s/it]

Average Metric: 18 / 20  (90.0%)
Score: 90.0 for set: [3]
Scores so far: [95.0, 90.0]
Best score: 95.0
2 candidate programs found.


After this step we have our examples ready, and we can test Gemini Pro on the same test dataset as above.

In [101]:
dspy.settings.configure(lm=dspy_gemini_pro, timeout=0)

In [104]:
evaluate(cot_fewshot)

Average Metric: 18 / 20  (90.0): 100%|██████████| 20/20 [00:07<00:00,  2.65it/s]

Average Metric: 18 / 20  (90.0%)



/Users/heikohotz/.pyenv/versions/3.11.7/envs/venv-ape/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['✔️ [True]' '✔️ [True]' '✔️ [True]' 'False' '✔️ [True]' '✔️ [True]'
 '✔️ [True]' '✔️ [True]' '✔️ [True]' '✔️ [True]' '✔️ [True]' '✔️ [True]'
 '✔️ [True]' '✔️ [True]' 'False' '✔️ [True]' '✔️ [True]' '✔️ [True]'
 '✔️ [True]' '✔️ [True]']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,gold_reasoning,example_answer,rationale,pred_answer,gsm8k_metric
0,"Amber, Micah, and Ahito ran 52 miles in total. Amber ran 8 miles. Micah ran 3.5 times what Amber ran. How many miles did Ahito...",Amber ran <<8=8>>8 miles. Micah ran 3.5 * 8 miles = <<3.5*8=28>>28 miles Ahito ran the rest = 52 - 8 - 28 = <<52-8-28=16>>16...,16,"find out how many miles Ahito ran. First, let's find out how many miles Micah ran: 3.5 x 8 = 28 miles. Now, let's add...",16,✔️ [True]
1,"Miguel uses 2 pads of paper a week for his drawing. If there are 30 sheets of paper on a pad of paper, how many...","Miguel uses 30 x 2 = <<30*2=60>>60 sheets of paper every week. Therefore, he uses 60 x 4 = <<60*4=240>>240 sheets of paper every month.",240,"find the number of sheets of paper Miguel uses every month. First, we need to find the total number of pads of paper he uses...",240,✔️ [True]
2,"At a certain grade level, three-fourths of students have a desktop computer at home. If 20 students do not have desktop computers, how many students...","Twenty students represent 1 - 3/4 = 1/4 of the students at that level. So, there are 4 x 20 students = <<4*20=80>>80 students at...",80,"find the total number of students. First, we need to find the fraction of students who have desktop computers: 1 - 3/4 = 1/4. Now,...",80,✔️ [True]
3,Comet Halley orbits the sun every 75 years. Bill's dad saw the Comet when he was 30 years old. Bill saw the comet a second...,Bill saw the Comet for the second time when he was 30 years * 3= <<30*3=90>>90 years old. Comet Halley can be seen every 75...,15,"find Bill's age when he saw the Comet. First, we need to find the number of years that passed between the two sightings: 75 years...",90,False
4,Tom plants 10 trees a year. Every year he also chops down 2 trees a year. He starts with 50 trees. After 10 years 30%...,He gets 10-2=<<10-2=8>>8 new trees a year After 10 years he has 8*10=<<8*10=80>>80 new trees So he had 80+50=<<80+50=130>>130 trees He lost 130*.3=<<130*.3=39>>39 trees That...,91,"find out how many trees Tom has left. First, let's find out how many trees Tom plants in 10 years: 10 trees/year x 10 years...",91,✔️ [True]
5,"John picks 4 bananas on Wednesday. Then he picks 6 bananas on Thursday. On Friday, he picks triple the number of bananas he did on...","Combining Wednesday and Thursday, John has 4 bananas + 6 bananas = <<4+6=10>>10 bananas. On Friday, he picks 3 * 4 bananas = <<3*4=12>>12 bananas....",22,"find the total number of bananas John has. First, let's find the number of bananas John picked on Friday: 4 x 3 = 12 bananas....",22,✔️ [True]
6,"Peyton scheduled after-work activities of a one hour yoga class on Monday, a cooking class that lasts three times as long as Monday’s yoga on...",Peyton’s cooking class will last 3 * 1 = <<3*1=3>>3 hours. The museum tour will take 3 / 2 = 1 1/2 hours. Peyton’s after-work...,8,calculate the total hours of Peyton's after-work activities. 1. **Yoga class (Monday):** 1 hour 2. **Cooking class (Tuesday):** 3 x 1 hour = 3 hours...,8,✔️ [True]
7,"Ben has 4 tubes of blue paint and 3 tubes of yellow paint. Jasper has half as many tubes of blue paint as Ben, and...",Jasper has 4/2= <<4/2=2>>2 tubes of blue paint Jasper has 3*3=<<3*3=9>>9 tubes of yellow paint Jasper has a total of 2+9 =<<2+9=11>>11 tubes of paint,11,"find out how many tubes of paint Jasper has. First, let's find out how many tubes of blue paint Jasper has: 4 / 2 =...",11,✔️ [True]
8,"Elaina is holding the final concert in her tour. To celebrate her final concert, she makes the concert twice as long as her usual concerts....","The concert, minus the encore, lasted for 65-minute concert – 15-minute encore = <<65-15=50>>50 minutes. This is twice as long as her usual concerts so...",25,"find the runtime of Elaina's usual concerts. First, we need to subtract the encore's runtime from the total runtime of the final concert: 65 -...",25,✔️ [True]
9,"Hannah slips on a banana peel and breaks her arm. The doctor charges 

90.0

Nice, this improved Gemini Pro's performance significantly from 35% :)

In [106]:
dspy_gemini_pro.inspect_history(n=1)





Answer math problems with numbers or short phrases.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: Usually a number or short phrase.

---

Question: A tank contains 6000 liters of water, 2000 liters evaporated, and then 3500 liters were drained by Bob. How many liters are in the tank if it now rains for 30 minutes and every 10 minutes 350 liters of rain are added to the tank?
Reasoning: Let's think step by step in order to find the total amount of water in the tank. First, 2000 liters evaporated from the initial 6000 liters, leaving 6000 - 2000 = 4000 liters. Next, Bob drained 3500 liters, resulting in 4000 - 3500 = 500 liters. Since it rained for 30 minutes, and 350 liters are added every 10 minutes, a total of 350 x 3 = 1050 liters were added. Therefore, the final amount of water in the tank is 500 + 1050 = 1550 liters.
Answer: 1550

---

Question: Louise is baking cakes for a gatheri